## Assignment 9.2

In [1]:
import os
import shutil
import json
from pathlib import Path

import pandas as pd

from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin.new_topic import NewTopic
from kafka.errors import TopicAlreadyExistsError

from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark import SparkConf
from pyspark.sql.functions import window, from_json, col
from pyspark.sql.types import StringType, TimestampType, DoubleType, StructField, StructType
from pyspark.sql.functions import udf

current_dir = Path(os.getcwd()).absolute()
checkpoint_dir = current_dir.joinpath('checkpoints')
locations_windowed_checkpoint_dir = checkpoint_dir.joinpath('locations-windowed')

if locations_windowed_checkpoint_dir.exists():
    shutil.rmtree(locations_windowed_checkpoint_dir)

locations_windowed_checkpoint_dir.mkdir(parents=True, exist_ok=True)

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Bret',
    last_name='Young'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config['locations_topic'] = '{}-locations'.format(config['topic_prefix'])
config['accelerations_topic'] = '{}-accelerations'.format(config['topic_prefix'])
config['windowed_topic'] = '{}-windowed'.format(config['topic_prefix'])

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Bret',
 'last_name': 'Young',
 'client_id': 'YoungBret',
 'topic_prefix': 'YoungBret',
 'locations_topic': 'YoungBret-locations',
 'accelerations_topic': 'YoungBret-accelerations',
 'windowed_topic': 'YoungBret-windowed'}

### Create Topic Utility Function

The `create_kafka_topic` helps create a Kafka topic based on your configuration settings.  For instance, if your first name is *John* and your last name is *Doe*, `create_kafka_topic('locations')` will create a topic with the name `DoeJohn-locations`.  The function will not create the topic if it already exists. 

In [3]:
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))

create_kafka_topic('windowed')

Topic "YoungBret-windowed" already exists


**TODO:** This code is identical to the code used in 9.1 to publish acceleration and location data to the `LastnameFirstname-simple` topic. You will need to add in the code you used to create the `df_accelerations` dataframe. In order to read data from this topic, make sure that you are running the notebook you created in assignment 8 that publishes acceleration and location data to the LastnameFirstname-simple topic.

In [4]:
spark = SparkSession\
    .builder\
    .appName("Assignment09")\
    .getOrCreate()

df_locations = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("subscribe", config['locations_topic']) \
  .load()

## TODO: Add code to create the df_accelerations dataframe
df_accelerations = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("subscribe", config['accelerations_topic']) \
  .load()

The following code defines a Spark schema for location and acceleration data as well as a user-defined function (UDF) for parsing the location and acceleration JSON data. 

In [5]:
location_schema = StructType([
    StructField('offset', DoubleType(), nullable=True),
    StructField('id', StringType(), nullable=True),
    StructField('ride_id', StringType(), nullable=True),
    StructField('uuid', StringType(), nullable=True),
    StructField('course', DoubleType(), nullable=True),
    StructField('latitude', DoubleType(), nullable=True),
    StructField('longitude', DoubleType(), nullable=True),
    StructField('geohash', StringType(), nullable=True),
    StructField('speed', DoubleType(), nullable=True),
    StructField('accuracy', DoubleType(), nullable=True),
])

acceleration_schema = StructType([
    StructField('offset', DoubleType(), nullable=True),
    StructField('id', StringType(), nullable=True),
    StructField('ride_id', StringType(), nullable=True),
    StructField('uuid', StringType(), nullable=True),
    StructField('x', DoubleType(), nullable=True),
    StructField('y', DoubleType(), nullable=True),
    StructField('z', DoubleType(), nullable=True),
])

udf_parse_acceleration = udf(lambda x: json.loads(x.decode('utf-8')), acceleration_schema)
udf_parse_location = udf(lambda x: json.loads(x.decode('utf-8')), location_schema)

See http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#window-operations-on-event-time for details on how to implement windowed operations. 

The following code selects the `timestamp` column from the `df_locations` dataframe that reads from the `LastnameFirstname-locations` topic and parses the binary value using the `udf_parse_location` UDF and defines the result to the `json_value` column.

```python
df_locations \
  .select(
    col('timestamp'), 
    udf_parse_location(df_locations['value']).alias('json_value')
  )
```

From here, you can select data from the `json_value` column using the `select` method. For instance, if you saved the results of the previous code snippet to `df_locations_parsed` you could select columns from the `json_value` field and assign them aliases using the following code. 

```python
df_locations_parsed.select(
    col('timestamp'), 
    col('json_value.ride_id').alias('ride_id'),
    col('json_value.uuid').alias('uuid'),
    col('json_value.speed').alias('speed')
  )
```

Next, you will want to add a watermark and group by `ride_id` and `speed` using a window duration of *30 seconds* and a slide duration of *15 seconds*. Use the `withWatermark` method in conjunction with the `groupBy` method. The [Spark streaming documentation](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#window-operations-on-event-time) should provide examples of how to do this. 

Next use the `mean` aggregation method to compute the average values and rename the column `avg(speed)` to `value` and the column `ride_id` to `key`. The reason you are renaming these values is that the PySpark Kafka API expects `key` and `value` as inputs. In a production example, you would setup serialization that would handle these details for you. 

When you are finished, you should have a streaming query with `key` and `value` as columns.

In [6]:
df_locations_parsed = df_locations.select(
    col('timestamp'), 
    udf_parse_location(df_locations['value']).alias('json_value')
  )

In [7]:
display(df_locations_parsed)

DataFrame[timestamp: timestamp, json_value: struct<offset:double,id:string,ride_id:string,uuid:string,course:double,latitude:double,longitude:double,geohash:string,speed:double,accuracy:double>]

In [8]:
df_locations_1 = df_locations_parsed \
    .select(
        col('timestamp'), 
        col('json_value.ride_id').alias('ride_id'),
        col('json_value.speed').alias('speed')
      ) 

In [9]:
display(df_locations_1)

DataFrame[timestamp: timestamp, ride_id: string, speed: double]

In [10]:
windowedSpeeds =  df_locations_1 \
    .withWatermark("timestamp", "30 seconds") \
    .groupBy(window("timestamp","30 seconds", "15 seconds"), "ride_id", "speed") \
    .mean('speed') \
    .withColumnRenamed("ride_id", "key") \
    .withColumnRenamed("avg(speed)", "value")

windowedSpeeds

DataFrame[window: struct<start:timestamp,end:timestamp>, key: string, speed: double, value: double]

In [11]:
windowedSpeeds.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- speed: double (nullable = true)
 |-- value: double (nullable = true)



In [12]:
import time

windowedSpeeds \
    .writeStream \
    .queryName("counting_1") \
    .format("memory") \
    .start()

In [14]:
for x in range(2):
  spark.sql("select key, value from counting_1").show()
  time.sleep(10)

+---+-----+
|key|value|
+---+-----+
+---+-----+

+---+-----+
|key|value|
+---+-----+
+---+-----+



In [27]:
print(windowedSpeeds.isStreaming)

True


In the previous Jupyter cells, you should have created the `windowedSpeeds` streaming query.  Next, you will need to write that to the `LastnameFirstname-windowed` topic. If you created the `windowsSpeeds` streaming query correctly, the following should publish the results to the `LastnameFirstname-windowed` topic. 

In [28]:
ds_locations_windowed = windowedSpeeds \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("topic", config['windowed_topic']) \
  .option("checkpointLocation", str(locations_windowed_checkpoint_dir)) \
  .start()

try:
    ds_locations_windowed.awaitTermination()
except KeyboardInterrupt:
    print("STOPPING STREAMING DATA")

StreamingQueryException: Writing job aborted.
=== Streaming Query ===
Identifier: [id = e3aa5b30-de48-4202-91b2-f4a822313a62, runId = 9e265553-bb7e-4f3e-8266-8f3b127672af]
Current Committed Offsets: {KafkaV2[Subscribe[YoungBret-locations]]: {"YoungBret-locations":{"0":1067}}}
Current Available Offsets: {KafkaV2[Subscribe[YoungBret-locations]]: {"YoungBret-locations":{"0":1078}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource org.apache.spark.sql.kafka010.KafkaStreamingWrite@42a646fe
+- Project [cast(key#6230 as string) AS key#8912, cast(value#6235 as string) AS value#8913]
   +- Project [window#6219-T30000ms, key#6230, speed#6213, avg(speed)#6224 AS value#6235]
      +- Project [window#6219-T30000ms, ride_id#6212 AS key#6230, speed#6213, avg(speed)#6224]
         +- Aggregate [window#6225-T30000ms, ride_id#6212, speed#6213], [window#6225-T30000ms AS window#6219-T30000ms, ride_id#6212, speed#6213, avg(speed#6213) AS avg(speed)#6224]
            